In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [ ]:
%run gptautomator/chat_gpt_automator.py
%run gptautomator/chat_gpt_session.py
from gptautomator.chat_gpt_chrome import ChatGPTChrome


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
chat_gpt = ChatGPTChrome()

Closing the browser...
You need to manually complete the log-in or the human verification if required.



DevTools listening on ws://127.0.0.1:53812/devtools/browser/0722c5cd-2114-4b96-9970-559854afba5b
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Continuing with the automation process...


In [10]:
chat_gpt.send_prompt_to_chatgpt("Hello, ChatGPT!")

In [11]:

class ChatGPTSession:
    def __init__(self, gpt):
        self.messages = []
        self.gpt = gpt
    
    def is_any_conv(self):
        return len(self.gpt.return_chatgpt_conversation()) > 0
    
    def ask(self, prompt):
        self._submit_prompt(prompt['value'])
        time.sleep(1)
        while True:
            send_buttons = self.gpt.driver.find_elements(By.CSS_SELECTOR,'button[data-testid="send-button"]')
            
            if len(send_buttons) > 0:
                return self.gpt.return_last_response()
            time.sleep(1)

    

    def _submit_prompt(self, value):
        self.gpt.send_prompt_to_chatgpt(value)
        

In [12]:

import csv
from io import StringIO


def get_page_tab_navigated_elements(driver):
    prev_focused_element = None
    initial_element = driver.find_element(By.TAG_NAME,"body")
    # List to store data related to focused elements
    data = []
    elements = []
    visited_elements = set()
    
    headers = ['ELEMENT_INDEX','TAG','TEXT', 'ID','NAME','INPUT_TYPE','VALUE','PLACEHOLDER','HREF', "XPATH"]
    for i in range(50):
        time.sleep(1)
        initial_element.send_keys(Keys.TAB)
        # Get the currently focused element
        focused_element = driver.switch_to.active_element
        elements.insert(i,focused_element)
        # Extract relevant information
        tag_name = focused_element.tag_name
        class_name = focused_element.get_attribute("class")
        input_value = focused_element.get_attribute("value")
        id = focused_element.get_attribute("id")
        placeholder = focused_element.get_attribute("placeholder")
        name = focused_element.get_attribute("name")
        text = focused_element.text if tag_name != "body" else None
        input_type = focused_element.get_attribute("type") if tag_name == "input" else None
        href = focused_element.get_attribute("href") if tag_name == "a" else None
        #xpath = driver.execute_script("return arguments[0].getXPath();", focused_element)

        data.append({
            "ELEMENT_INDEX": i,
            "TAG": tag_name,
            "TEXT": text,
            "ID": id,
            "NAME": name,
            "VALUE": input_value,
            "PLACEHOLDER": placeholder,
            "INPUT_TYPE": input_type,
            "HREF": href,
           # "XPATH": xpath
        })
        text_small = focused_element.text[:10] if tag_name != "body" else None
        element_identifier = (tag_name,id,text_small,class_name)
        if element_identifier in visited_elements:
            break  # Exit loop if the element has been visited before
        else:
            visited_elements.add(element_identifier)

     
    csv_buffer = StringIO()
    writer = csv.DictWriter(csv_buffer, fieldnames=headers)
    writer.writeheader()
    for item in data:
        writer.writerow(item)

    # Get CSV data as a string
    csv_data = csv_buffer.getvalue()

    initial_element.send_keys(Keys.CONTROL + 'l')
    
    return csv_data, elements

In [13]:
get_xpath_script = """
function getXPath(element) {
    if (element.id !== '') {
        return 'id("' + element.id + '")';
    }
    if (element === document.body) {
        return element.tagName;
    }
    var ix = 0;
    var siblings = element.parentNode.childNodes;
    for (var i = 0; i < siblings.length; i++) {
        var sibling = siblings[i];
        if (sibling === element) {
            return getXPath(element.parentNode) + '/' + element.tagName + '[' + (ix + 1) + ']';
        }
        if (sibling.nodeType === 1 && sibling.tagName === element.tagName) {
            ix++;
        }
    }
}

// Define a global function to get XPath
window.getXPath = getXPath;
"""

In [14]:
p_driver = webdriver.Chrome()

In [16]:
p_driver.get("https://amazon.jobs/en-gb/")
time.sleep(1)

In [17]:
result = get_page_tab_navigated_elements(p_driver)
page_elements_data, elements = result

In [18]:
print(page_elements_data)

ELEMENT_INDEX,TAG,TEXT,ID,NAME,INPUT_TYPE,VALUE,PLACEHOLDER,HREF,XPATH
0,div,"Notice of Right to Opt-Out Sharing
Amazon Jobs uses cookies and similar tools such as pixels (collectively, “cookies”) for advertising purposes. The cookies collect and share information about you with third-parties to serve you with relevant advertisements and job recommendations based on your interactions with Amazon Jobs and third-party sites. California residents may exercise their right to opt-out of the sharing of information for advertising purposes by clicking toggling the buttons below. You can also manage your cookie preferences through the Cookie Preference Center.
For more information about how we use cookies and information of how we collect cookies, please visit our Candidate Cookie Notice and Candidate Privacy Notice and the California supplement notice.",,,,,,,
1,button,Accept all,btn-accept-cookies,,,,,,
2,button,Reject all,btn-reject-cookies,,,,,,
3,div,Customize cookies,,,,,,,
4,a,Customize

In [17]:
p_driver.quit()

In [26]:
import json

task_details = """ 
User Preferences for the application of job
Locations: New York, Canada
Job titles : Senior Software Engineer, Backend Developer

Previously Completed Preferences:
Location, Job Title
New York, Backend Developer
Canada,Senior Software Engineer


Given below is the elements navigated with tab complete page and all elements of the webpage in CSV. The task is ANSWERS the QUESTIONS. 
The task is to apply for jobs in that given web page. You need to help me out to take actions using chrome selenium commands.
Indices are references to selenium elements array. For before click commands, maintain sleep time of 2 seconds.

Example: Lets say if I have search for jobs and select the location. Give response like below JSON FORMAT ONLY so that I can automate my tasks
    {
        "actions : ["elements[4].send_keys("Backend Developer")",
                            "elements[5].send_keys("New York")",
                            "time.sleep(2)", 
                            "elements[10].click()"]
        "job_preferences": {
            "location" : "New York",
            "job_title" : "Backend Developer"
        }

DO NOT ADD ANY COMMENTS IN JSON AS I HAVE TO PARSE THE RESPONSE FOR USAGE. PLEASE KEEP IN MIND

If there is any accept cookies to perform if present in page focus 
then add that action to the top example command to accept all cookies and add time.sleep(1)
otherwise please DON'T ADD
Eg:
 add "elements[2].click()",
 add "time.sleep(1)"

WebPage Related Information

Tab Navigated Elements CSV
    
"""
prompt = f"""{task_details}
    {page_elements_data}
"""
#prompt = prompt.replace("\n", "\\n")


In [21]:
gptSession = ChatGPTSession(chat_gpt)

In [27]:
gpt_response = gptSession.ask({
    "value" : json.dumps(prompt)
})

gpt_resp_dict = json.loads(gpt_response)


In [25]:
print(gpt_response)

{
    "actions": [
        "elements[1].click()",  // Accept all cookies
        "time.sleep(1)",  // Sleep for 1 second after accepting cookies
        "elements[9].send_keys(\"Backend Developer\")",  // Input job title
        "elements[10].send_keys(\"New York\")",  // Input location
        "time.sleep(2)",  // Sleep for 2 seconds
        "elements[11].click()"  // Click the search button
    ],
    "job_preferences": {
        "location": "New York",
        "job_title": "Backend Developer"
    }
}


In [32]:
actions = gpt_resp_dict['actions']
exec(actions[3])

[6787:36611:0414/205524.360289:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0414/212757.207270:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0414/220443.138865:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0414/224034.175298:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0414/234115.541176:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0415/001636.673283:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0415/012046.528335:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0415/020115.588018:ERROR:connection_factory_impl.cc(483)] ConnectionHandler failed with net error: -2
[6787:36611:0415/023914.613501:ERROR:connection_factory_impl.cc(483)] ConnectionHandler 

In [18]:
chat_gpt.quit()

Closing the browser...


In [30]:
elements[0]

<selenium.webdriver.remote.webelement.WebElement (session="ab3321edcafecd6455e417e6a553382c", element="f.83B4A9C52C3C3A317B2D63203E45D186.d.F6B908712003119424887C34AD63034D.e.346")>